In [1]:
from datetime import time
from collections import Counter
import requests
from selenium.webdriver.firefox import webdriver
%run ParsePatent.ipynb
from selenium import webdriver
import time
import pandas as pd
import xlrd
import xlwt
from xlwt import Workbook


def TopGenesInPatents(targets):

    my_dict = {i:targets.count(i) for i in targets}
    sor = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
    sor = list(sor)

    targetList = []
    amountList = []
    total = 0
    for target in sor:
        target = list(target)
        targetList.append(target[0])
        amountList.append(target[1])
        total += target[1]

    df = pd.DataFrame.from_dict({'Column1': targetList, 'Column2': amountList})
    writer = pd.ExcelWriter('Popular2.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Genes', index=False)
    writer.save()

    return sor

def RunningCompanyData():
    patents = PatentsFromFile()
    #patents = testFile.manual_patents

    return testmanual(patents)

def RunningCompanyFinal(info):
    diseases = diseasesfromfile()

    return TopGenesInPatents(info)

def testmanual(patents):
    newPatents = []

    for patent in patents:
        patent = str(patent)
        newPatent = patent.replace('-', '')

        if len(str(newPatent)) == 14:
            newPatent = newPatent[0:6] + "0" + newPatent[6:]
        newPatents.append(newPatent)

    wb = Workbook()
    sheet1 = wb.add_sheet('Sheet 1')
    i = 0
    driver = webdriver.Firefox(
        executable_path=r"C:\Python\Python38\geckodriver.exe")

    aliases = newaliasesfromfile()
    genes = genesfromfile()
    firstlist = {}

    while i < len(newPatents):
        driver.get("https://patents.google.com/patent/" + newPatents[i])
        time.sleep(5)

        targets = parse(driver, aliases, genes, 3, 10)
        k = 0

        if len(targets) > 13:
            i = i + 1
            continue

        sheet1.write(i + 1, 0, newPatents[i])

        for target in targets:
            sheet1.write(i + 1, k + 1, target)
            k = k + 1
        wb.save('CompanyData.xls')

        firstlist[newPatents[i]] = targets
        print(i)
        i = i + 1

    driver.close
    return firstlist

def newaliasesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    # print(pepega["Unnamed: 0"].values[0])
    # print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog]) - 1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers


def genesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    return pepega["Unnamed: 0"]


def diseasesfromfile():
    pepega = pd.read_excel("Gene_Conditions_All.xlsx")
    # print(pepega["Unnamed: 0"].values[0])
    # print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog]) - 1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers


def diseasesFromTarget(foundTarget, diseases):

    for target in diseases:
        if target[0] == foundTarget:
            return target[1:]
        
def PatentsFromFile():
    pepega = pd.read_excel("Celgene_Patent_Data.xls")
    firstColumn = pepega["search URL:"]
    firstColumn.pop(0)

    return firstColumn

def CompanyGenesFromFile():
    pepega = pd.read_excel(r"C:\Users\zaids\Downloads\NovartisData.xls")
    firstColumn = list(pepega["Unnamed: 1"]) + list(pepega["Unnamed: 2"]) + list(pepega["Unnamed: 3"]) + list(pepega["Unnamed: 4"]) + list(pepega["Unnamed: 5"]) + list(pepega["Unnamed: 6"]) + list(pepega["Unnamed: 7"]) + list(pepega["Unnamed: 8"]) + list(pepega["Unnamed: 9"]) + list(pepega["Unnamed: 10"])

    newlist = [x for x in firstColumn if pd.isnull(x) == False and x != 'nan']

    return newlist